# Problem 1: Used Cars

In [176]:
# importing libraries
import pandas as pd
import numpy as np

# load the dataset
df = pd.read_csv('../data/raw/used_car_data_raw.csv', index_col=0)
df.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13 km/kg,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74
6,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08 kmpl,1461 CC,63.1 bhp,5.0,NaN,3.50


In [177]:
# check data info and for missing values
df.info()
print("\nMissing Values:", df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
Index: 5847 entries, 1 to 6018
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               5847 non-null   object 
 1   Location           5847 non-null   object 
 2   Year               5847 non-null   int64  
 3   Kilometers_Driven  5847 non-null   int64  
 4   Fuel_Type          5847 non-null   object 
 5   Transmission       5847 non-null   object 
 6   Owner_Type         5847 non-null   object 
 7   Mileage            5845 non-null   object 
 8   Engine             5811 non-null   object 
 9   Power              5811 non-null   object 
 10  Seats              5809 non-null   float64
 11  New_Price          815 non-null    object 
 12  Price              5847 non-null   float64
dtypes: float64(2), int64(2), object(9)
memory usage: 639.5+ KB

Missing Values: Name                    0
Location                0
Year                    0
Kilometers_Driven  

## A) Handle Missing Values

### Strategy
- Mileage, Engine, Power, Seats: Impute with Median or Mode.
- New_Price: This column has a ton of missing values.
- Location, Fuel_Type, etc: Check for any missing categorical data.

In [178]:
# calculate percentage of missing values
missing_percent = df.isnull().sum() / len(df) * 100
print(missing_percent)

Name                  0.000000
Location              0.000000
Year                  0.000000
Kilometers_Driven     0.000000
Fuel_Type             0.000000
Transmission          0.000000
Owner_Type            0.000000
Mileage               0.034206
Engine                0.615700
Power                 0.615700
Seats                 0.649906
New_Price            86.061228
Price                 0.000000
dtype: float64


### Justification:
- New_Price has a very high percentage of missing values. Imputing it would introduce too much bias. I will drop it.
- Impute Seats with Mode and Mileage, Engine, Power (after cleaning) with Median.

- Note: Need to clean units first before imputing numerical means/medians for Mileage, Engine, Power.
- Handle missing values for these AFTER cleaning units in Part B, or drop rows if minimal.

In [179]:
# handle missing values
# 1. Seats: impute with mode
df['Seats'].fillna(df['Seats'].mode()[0], inplace=True)

# 2. New_Price: drop due to excessive missing values
# drop this column to avoid introducing significant bias
if 'New_Price' in df.columns:
    df.drop('New_Price', axis=1, inplace=True)
    print("Dropped 'New_Price' column.")

# note: mileage, engine, and power missing values will be handled
# after unit removal and conversion to numeric types in part b

Dropped 'New_Price' column.


## B) Remove Units and Separate Mileage

### Strategy
- Split Mileage into Mileage_kmpl and Mileage_kmkg columns.
- Remove CC from Engine.
- Remove bhp from Power.
- Remove Lakh from New_Price (if it still exists).

In [180]:
# function to extract numeric part
def clean_currency(x):
    if isinstance(x, str):
        return float(x.split(' ')[0])
    return x

# 1. Mileage: split into two columns (kmpl and km/kg)
# create new columns initialized with nan
df['Mileage_kmpl'] = np.nan
df['Mileage_kmkg'] = np.nan

# fill based on unit
mask_kmpl = df['Mileage'].str.endswith('kmpl', na=False)
mask_kmkg = df['Mileage'].str.endswith('km/kg', na=False)

# extract numeric values
df.loc[mask_kmpl, 'Mileage_kmpl'] = df.loc[mask_kmpl, 'Mileage'].apply(lambda x: float(str(x).split(' ')[0]))
df.loc[mask_kmkg, 'Mileage_kmkg'] = df.loc[mask_kmkg, 'Mileage'].apply(lambda x: float(str(x).split(' ')[0]))

# drop original mileage column
df.drop('Mileage', axis=1, inplace=True)

# 2. Engine: remove cc and convert to float
df['Engine'] = df['Engine'].str.replace(' CC', '').astype(float)

# 3. Power: remove bhp and handle null
df['Power'] = df['Power'].str.replace(' bhp', '')
df['Power'] = pd.to_numeric(df['Power'], errors='coerce')

# 4. Impute missing values for numerical columns with median
df['Engine'].fillna(df['Engine'].median(), inplace=True)
df['Power'].fillna(df['Power'].median(), inplace=True)
df['Mileage_kmpl'].fillna(df['Mileage_kmpl'].median(), inplace=True)
df['Mileage_kmkg'].fillna(df['Mileage_kmkg'].median(), inplace=True)

df.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Engine,Power,Seats,Price,Mileage_kmpl,Mileage_kmkg
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,1582.0,126.20,5.0,12.50,19.67,13.0
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,1199.0,88.70,5.0,4.50,18.19,13.0
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,1248.0,88.76,7.0,6.00,20.77,13.0
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,1968.0,140.80,5.0,17.74,15.20,13.0
6,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,1461.0,63.10,5.0,3.50,23.08,13.0


## C) One-Hot Encoding

### Strategy
- Convert Fuel_Type and Transmission into numerical one-hot encoded values using get_dummies.

In [181]:
# one-hot encode categorical variables
df = pd.get_dummies(df, columns=['Fuel_Type', 'Transmission'], drop_first=True)
df.head()

,Name,Location,Year,Kilometers_Driven,Owner_Type,Engine,Power,Seats,Price,Mileage_kmpl,Mileage_kmkg,Fuel_Type_Electric,Fuel_Type_Petrol,Transmission_Manual
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,First,1582.0,126.20,5.0,12.50,19.67,13.0,False,False,True
2,Honda Jazz V,Chennai,2011,46000,First,1199.0,88.70,5.0,4.50,18.19,13.0,False,True,True
3,Maruti Ertiga VDI,Chennai,2012,87000,First,1248.0,88.76,7.0,6.00,20.77,13.0,False,False,True
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Second,1968.0,140.80,5.0,17.74,15.20,13.0,False,False,False
6,Nissan Micra Diesel XV,Jaipur,2013,86999,First,1461.0,63.10,5.0,3.50,23.08,13.0,False,False,True


## D) Feature Engineering

### Strategy
- Create Current_Age by subtracting Year from the current year (2025).

In [182]:
# calculate current age
current_year = 2025
df['Current_Age'] = current_year - df['Year']
df[['Year', 'Current_Age']].head()

,Year,Current_Age
1,2015,10
2,2011,14
3,2012,13
4,2013,12
6,2013,12


## E) Data Operations

### Strategy
- Perform the Select, Filter, Rename, Mutate, Arrange, and Summarize operations.

In [183]:
# 1. Select: specific columns
selected_df = df[['Name', 'Price', 'Current_Age']]

# 2. Filter: cars with price > 10 lakhs
filtered_df = df[df['Price'] > 10]

# 3. Rename: price to price_lakhs
df.rename(columns={'Price': 'Price_Lakhs'}, inplace=True)

# 4. Mutate: create price_per_year
df['Price_Per_Year'] = df['Price_Lakhs'] / df['Current_Age']

# 5. Arrange: sort by price_lakhs descending
sorted_df = df.sort_values(by='Price_Lakhs', ascending=False)

# 6. Summarize: average price by location
summary_df = df.groupby('Location')['Price_Lakhs'].mean().reset_index()

print("Summary by Location:")
print(summary_df)

Summary by Location:
      Location  Price_Lakhs
0    Ahmedabad     8.567248
1    Bangalore    13.482670
2      Chennai     7.958340
3   Coimbatore    15.160206
4        Delhi     9.881944
5    Hyderabad     9.997423
6       Jaipur     5.916725
7        Kochi    11.309109
8      Kolkata     5.733924
9       Mumbai     9.592546
10        Pune     6.951000


## Export Processed Data

In [184]:
df.to_csv('../data/processed/used_car_data_clean.csv', index=False)
print("Data Saved!")

Data Saved!
